# Anime Recommendation System

### Dataset: https://www.kaggle.com/datasets/dbdmobile/myanimelist-dataset

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from rapidfuzz import fuzz

In [2]:
anime_df = pd.read_csv('anime-filtered.csv')

In [3]:
anime_df.head()

,anime_id,Name,Score,Genres,English name,Japanese name,sypnopsis,Type,Episodes,Aired,...,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,"In the year 2071, humanity has colonized sever...",TV,26,"Apr 3, 1998 to Apr 24, 1999",...,24 min. per ep.,R - 17+ (violence & profanity),28.0,39,1251960,61971,105808,718161,71513,26678
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,"other day, another bounty—such is the life of ...",Movie,1,"Sep 1, 2001",...,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518,273145,1174,4143,208333,1935,770
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,"Vash the Stampede is the man with a $$60,000,0...",TV,26,"Apr 1, 1998 to Sep 30, 1998",...,24 min. per ep.,PG-13 - Teens 13 or older,266.0,201,558913,12944,29113,343492,25465,13925
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),ches are individuals with special powers like ...,TV,26,"Jul 2, 2002 to Dec 24, 2002",...,25 min. per ep.,PG-13 - Teens 13 or older,2481.0,1467,94683,587,4300,46165,5121,5378
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,It is the dark century and the people are suff...,TV,52,"Sep 30, 2004 to Sep 29, 2005",...,23 min. per ep.,PG - Children,3710.0,4369,13224,18,642,7314,766,1108


In [4]:
anime_df.drop('anime_id', axis=1, inplace=True)
anime_df.reset_index(drop=False, inplace=True)
anime_df.rename(columns={'index': 'anime_id'}, inplace=True)

In [5]:
features = ['Score', 'Genres', 'Studios', 'Producers', 'Episodes', 'Source']

In [6]:
for feature in features:
    anime_df[feature] = anime_df[feature].astype('str')
    anime_df[feature] = anime_df[feature].fillna('')

In [7]:
def combined_features(row)-> str:
    return row['Genres']+" "+row['Score']+" "+row['Studios']+" "+row['Producers']+" "+row['Source']+" "+row['Episodes']

In [8]:
def get_index_from_anime(name: str) -> str:

    anime_df['ratio_normal'] = anime_df['Name'].apply(lambda row: fuzz.ratio(row, name))
    highest_ratio_row_1 = anime_df.loc[anime_df['ratio_normal'].idxmax()]
    
    anime_df['ratio_english'] = anime_df['English name'].apply(lambda row: fuzz.ratio(row, name))
    highest_ratio_row_2 = anime_df.loc[anime_df['ratio_english'].idxmax()]
    
    row1 = fuzz.ratio(highest_ratio_row_1['Name'], name)
    row2 = fuzz.ratio(highest_ratio_row_2['English name'], name)
    
    return highest_ratio_row_1['anime_id'] if row1 > row2 else highest_ratio_row_2['anime_id']

In [9]:
def get_recommendation(anime_name: str) -> list:
    
    anime_index = get_index_from_anime(anime_name)
    similar_animes = list(enumerate(cosine_sim[anime_index]))
    sorted_similar_animes = sorted(similar_animes, key = lambda x:x[1], reverse = True)[:100]
    
    result = []
    i=0
    for anime_ref in sorted_similar_animes:
        if i >= 10:
            break
        anime_name_in_df = anime_df[anime_df.index == anime_ref[0]]["Name"].values[0]
        
        if anime_name.split(" ")[0] == anime_name_in_df.split(" ")[0]:
            continue
        
        result.append(anime_name_in_df)
        i+=1
        
    return result

In [10]:
anime_df['combined_features'] = anime_df.apply(combined_features,axis = 1)

In [11]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(anime_df['combined_features'])

In [12]:
cosine_sim = cosine_similarity(count_matrix)

In [13]:
your_anime = "Jujutsu Kaisen"
top_ten_recommendations = get_recommendation(your_anime)
top_ten_recommendations

['Stop!! Hibari-kun!',
 'Air Gear Special',
 'Kindaichi Shounen no Jikenbo: Kuromajutsu Satsujin Jiken-hen',
 'Tiger Mask Fuku Men League Sen',
 'Kindaichi Shounen no Jikenbo: Shinigami Byouin Satsujin Jiken',
 'One Piece Film: Gold',
 'Hokuto no Ken Movie',
 'One Piece: Jango no Dance Carnival',
 'One Piece: Straw Hat Theater',
 'Sekishoku Elegy']